# Fine-tuning Gemma-2B for Smart Contact Lists

This notebook fine-tunes Google's Gemma-2B model on the smart contact lists training data to generate FredQL queries.

## 📋 Prerequisites
1. **Upload Training Data**: Upload `smart_lists_training.csv` to the root `/content/` directory
2. **HuggingFace Token**: Add your HF token to Colab secrets as `HF_TOKEN`
3. **GPU Runtime**: Enable T4 GPU (Runtime → Change runtime type → GPU)

## ⚙️ Setup Instructions

### Step 1: Enable GPU
1. Go to **Runtime** → **Change runtime type**
2. Select **Hardware accelerator**: **GPU** (T4)
3. Click **Save**

### Step 2: Upload Training Data
1. Click the **Files** icon (📁) on the left sidebar
2. Click **Upload** button
3. Upload `smart_lists_training.csv` to `/content/` directory

### Step 3: Set HuggingFace Token
1. Go to https://huggingface.co/settings/tokens
2. Create a new token with "Read" permission
3. In Colab, click the **🔑 Secrets** icon on the left
4. Add a new secret:
   - Name: `HF_TOKEN`
   - Value: Your HuggingFace token
5. Enable notebook access

### Step 4: Run All Cells
Click **Runtime** → **Run all** or run cells sequentially

---

## 📊 What This Notebook Does

1. **Installs Dependencies** (~2 min)
2. **Loads Gemma-2B** model with 4-bit quantization
3. **Loads Training Data** from `smart_lists_training.csv`
4. **Configures LoRA** for parameter-efficient fine-tuning
5. **Trains the Model** (~10-15 min for 100 steps)
6. **Tests the Model** with sample queries
7. **Saves & Downloads** the fine-tuned model

---


In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 29.8 MB/s eta 0:00:00


In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [ ]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

### Prerequisites
* nf4(4-bit NormalFloat(NF4)) : https://www.kaggle.com/code/lorentzyeung/what-s-4-bit-quantization-how-does-it-help-llama2


In [ ]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
text = "Quote: Imagination is more,"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more, than knowledge.

I am a self-taught artist, born in 1985 in


In [ ]:
text = "Quote: Imagination is more"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more important than knowledge. Knowledge is limited. Imagination encircles the world.

- Albert Einstein

The


In [ ]:
os.environ["WANDB_DISABLED"] = "false"

In [ ]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [ ]:
import pandas as pd
from datasets import Dataset

# Load CSV file from root directory
csv_path = "/content/smart_lists_training.csv"

# Check if file exists
import os
if not os.path.exists(csv_path):
    raise FileNotFoundError(
        f"❌ Training data not found at {csv_path}\n"
        "Please upload 'smart_lists_training.csv' to /content/ directory\n"
        "Use the Files panel (📁) on the left sidebar → Upload"
    )

# Load CSV into pandas
df = pd.read_csv(csv_path)
print(f"✓ Loaded {len(df)} training examples from CSV")
print(f"✓ Columns: {list(df.columns)}")

# Convert to HuggingFace Dataset
data = Dataset.from_pandas(df)

# Preview first example
print("\n📝 First training example:")
print(f"Instruction: {data['instruction'][0]}")
print(f"Response: {data['response'][0][:100]}...")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [ ]:
# Preview the dataset structure
print("Dataset features:", data.features)
print(f"Number of examples: {len(data)}")
print("\nFirst 3 examples:")
for i in range(min(3, len(data))):
    print(f"\n{i+1}. Instruction: {data['instruction'][i][:60]}...")
    print(f"   Response: {data['response'][i][:60]}...")

['“Be yourself; everyone else is already taken.”',
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”",
 '“So many books, so little time.”',
 '“A room without books is like a body without a soul.”',
 "“Be who you are and say what you feel, because those who mind don't matter, and those who matter don't mind.”",
 "“You've gotta dance like there's nobody watching,Love like you'll never be hurt,Sing like there's nobody listening,And live like it's heaven on earth.”",
 "“You know you're in love when you can't fall asleep because reality is finally better than your dreams.”",
 '“You only live once, but if you do it right, once is enough.”',
 '“Be the change that you wish to see in the world.”',
 "“In three words I can sum up everything I

In [ ]:
def formatting_func(example):
    """Format examples using Alpaca template"""
    texts = []
    for instruction, response in zip(example['instruction'], example['response']):
        text = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
{response}"""
        texts.append(text)
    return texts

In [ ]:
# Test the formatting function
sample = {
    'instruction': [data['instruction'][0]],
    'response': [data['response'][0]]
}
formatted_sample = formatting_func(sample)
print("📝 Formatted training example:")
print("="*80)
print(formatted_sample[0][:300] + "...")
print("="*80)

Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 2508
})

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data,  # Using our loaded dataset
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,  # Adjust based on dataset size
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="steps",
        save_steps=50,
        save_total_limit=2,
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
    max_seq_length=2048,  # Set max sequence length
)

print(f"✓ Trainer configured")
print(f"✓ Training on {len(data)} examples")
print(f"✓ Max steps: 100")
print(f"✓ Batch size: 1 (with gradient accumulation: 4)")
print(f"✓ Effective batch size: 4")

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
1,1.680200
2,0.629800
3,1.022100
4,1.030600
5,0.418500
6,1.227500
7,1.092000
8,0.332100
9,0.562600
10,0.479100


TrainOutput(global_step=100, training_loss=0.1457195576839149, metrics={'train_runtime': 56.7229, 'train_samples_per_second': 7.052, 'train_steps_per_second': 1.763, 'total_flos': 55030401331200.0, 'train_loss': 0.1457195576839149, 'epoch': 66.67})

In [ ]:
# Test 1: Find customers by email
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Find customers whose email contains gmail

### Response:
"""

device = "cuda:0"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7, do_sample=True)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("📝 Test 1: Find customers by email")
print("="*80)
print(response)
print("="*80)

Quote: A woman is like a tea bag; you can’t tell how strong she is until you put her in hot water
Author: Eleanor


In [ ]:
# Test 2: Find customers by recent booking
prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
List people who booked in the last 7 days

### Response:
"""

device = "cuda:0"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7, do_sample=True)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n📝 Test 2: Find customers by recent booking")
print("="*80)
print(response)
print("="*80)

Quote: Outside of a dog, a book is man's best friend.
Author: Nicolas Chamfort
Quote: The most wasted of all days is one


In [ ]:
# Save the fine-tuned model
output_dir = "./gemma-2b-smart-lists"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✓ Model saved to: {output_dir}")
print(f"✓ Files saved:")
print(f"  - adapter_model.bin")
print(f"  - adapter_config.json")
print(f"  - tokenizer files")
print("\n📦 To download, use Files panel (📁) or run:")
print("  from google.colab import files")
print("  import shutil")
print("  shutil.make_archive('gemma-2b-smart-lists', 'zip', output_dir)")
print("  files.download('gemma-2b-smart-lists.zip')")


In [ ]:
# Download the model (optional)
from google.colab import files
import shutil

# Create zip file
shutil.make_archive('gemma-2b-smart-lists', 'zip', './gemma-2b-smart-lists')

# Download
print("📥 Downloading model...")
files.download('gemma-2b-smart-lists.zip')
print("✓ Download complete!")
